In [1]:
import polars as pl
from data_source.make_dataset import load_gsheet_data
from data_source.sheet_ids import LOGISTICS_EFORM_ID, PTI_SHEET,WASHING_SHEET

PTI = await load_gsheet_data(LOGISTICS_EFORM_ID, PTI_SHEET)
WASHING = await load_gsheet_data(LOGISTICS_EFORM_ID,WASHING_SHEET)

In [8]:
washing_april = WASHING.select(
    pl.col("date").str.to_date(format="%d/%m/%Y"),
    pl.col("container_number"),
    pl.col("shipping_line"),
    pl.col("Invoice To")
).filter(
    pl.col("date").dt.month().eq(4).and_(
        pl.col("shipping_line").eq(pl.lit("MAERSK"))
    )
).collect()

In [9]:
washing_april.filter(pl.col("container_number").is_duplicated()).sort(by="container_number")

date,container_number,shipping_line,Invoice To
date,str,str,str
2025-04-05,"""MMAU1004670""","""MAERSK""","""MAERSKLINE"""
2025-04-30,"""MMAU1004670""","""MAERSK""",""""""
2025-04-15,"""MNBU0482570""","""MAERSK""","""MAERSKLINE"""
2025-04-22,"""MNBU0482570""","""MAERSK""",""""""
2025-04-26,"""MNBU0482713""","""MAERSK""",""""""
…,…,…,…
2025-04-29,"""MNBU4179310""","""MAERSK""",""""""
2025-04-17,"""MNBU4562165""","""MAERSK""",""""""
2025-04-25,"""MNBU4562165""","""MAERSK""",""""""


In [2]:
maersk_april = PTI.select(
    pl.col("Date Plugin")
    .str.to_date(format="%d/%m/%Y")
    .dt.combine(pl.col("Time Plugin").str.to_time(format="%H:%M:%S"))
    .alias("date_plug"),
    pl.col("Container Number").cast(pl.Utf8).alias("container_number"),
    pl.col("Set Point").alias("set_point"),
    pl.col("Shipping Line").alias("line"),
    pl.col("Unplugged Date")
    .str.to_datetime(format="%d/%m/%Y %H:%M", strict=False)
    .alias("date_unplug"),
    pl.col("Status"),
).filter(pl.col("date_plug").dt.month().eq(4).and_(
    pl.col("line").eq("MAERSK")
)).with_columns(
    hours = (pl.col("date_unplug") - pl.col("date_plug")).dt.total_seconds() / 3600
).collect()

In [4]:
maersk_april.filter(pl.col("date_unplug").is_null())

date_plug,container_number,set_point,line,date_unplug,Status,hours
datetime[μs],str,i64,str,datetime[μs],str,f64
2025-04-11 16:01:00,"""MNBU3672492""",-25,"""MAERSK""",null,"""""",null
2025-04-16 09:39:00,"""MNBU4103000""",-25,"""MAERSK""",null,"""""",null
2025-04-16 09:43:00,"""MNBU4103000""",-25,"""MAERSK""",null,"""""",null
2025-04-25 16:54:00,"""MNBU3083363""",-25,"""MAERSK""",null,"""""",null
2025-04-28 08:33:00,"""MNBU3134060""",-25,"""MAERSK""",null,"""""",null
2025-04-28 08:33:00,"""MNBU4179310""",-25,"""MAERSK""",null,"""""",null


In [5]:
maersk_april.filter(pl.col("hours").lt(0))

date_plug,container_number,set_point,line,date_unplug,Status,hours
datetime[μs],str,i64,str,datetime[μs],str,f64
2025-04-18 19:01:00,"""MNBU3613170""",-25,"""MAERSK""",2025-04-18 08:04:00,"""PASSED""",-10.95


In [11]:
maersk_april.filter(pl.col("hours").gt(30)).sort(by="hours")

date_plug,container_number,set_point,line,date_unplug,Status,hours
datetime[μs],str,i64,str,datetime[μs],str,f64
2025-04-06 04:00:00,"""MSFU0014439""",-60,"""MAERSK""",2025-04-07 11:29:00,"""PASSED""",31.483333
2025-04-18 19:30:00,"""MNBU4041450""",-25,"""MAERSK""",2025-04-20 10:12:00,"""FAILED""",38.7
2025-04-10 18:21:00,"""MNBU3672492""",-25,"""MAERSK""",2025-04-12 10:20:00,"""PASSED""",39.983333
2025-04-10 18:21:00,"""MNBU9151465""",-25,"""MAERSK""",2025-04-12 10:20:00,"""FAILED""",39.983333
2025-04-07 16:32:00,"""MNBU3069112""",-25,"""MAERSK""",2025-04-09 09:08:00,"""PASSED""",40.6
2025-04-26 08:13:00,"""MNBU4179310""",-25,"""MAERSK""",2025-04-28 15:55:00,"""FAILED""",55.7
2025-04-14 07:58:00,"""MNBU9151465""",-25,"""MAERSK""",2025-04-16 16:38:00,"""PASSED""",56.666667
2025-04-11 16:01:00,"""MNBU9151465""",-25,"""MAERSK""",2025-04-14 16:11:00,"""FAILED""",72.166667
2025-04-25 10:00:00,"""MNBU3134060""",-25,"""MAERSK""",2025-04-28 15:55:00,"""FAILED""",77.916667


In [13]:
maersk_april.filter(pl.col("date_plug").dt.hour().lt(7))

date_plug,container_number,set_point,line,date_unplug,Status,hours
datetime[μs],str,i64,str,datetime[μs],str,f64
2025-04-06 04:07:00,"""MSFU0012776""",-60,"""MAERSK""",2025-04-07 08:01:00,"""PASSED""",27.9
2025-04-06 04:00:00,"""MSFU0014439""",-60,"""MAERSK""",2025-04-07 11:29:00,"""PASSED""",31.483333


In [18]:
maersk_april.filter(pl.col("date_plug").dt.hour().gt(19))

date_plug,container_number,set_point,line,date_unplug,Status,hours
datetime[μs],str,i64,str,datetime[μs],str,f64


In [6]:
maersk_april.filter(pl.col("container_number").is_duplicated()).sort(by="container_number").write_clipboard()

In [1]:
import polars as pl
import pretty_errors
from datetime import date

from dataframe import netlist, stuffing
# from data_source.make_dataset import load_gsheet_data
# from data_source.sheet_ids import STUFFING_SHEET_ID,plugin_sheet

pretty_errors.activate()

vessel_name = "TXORI ZURI"
start_date = date(2025, 2, 12)
end_date = date(2025, 2, 15)


wDATA = await netlist.oss()

stuffingData = await stuffing.coa()

wDATA.filter(
    pl.col("vessel").eq(pl.lit(vessel_name)),
    pl.col("date").is_between(start_date, end_date),
).join(
    stuffingData.with_columns(
        container_number=pl.col("container_number").cast(pl.Utf8)
    ),
    left_on=["date", "destination"],
    right_on=["date_plugged", "container_number"],
    how="left",
).select(
    pl.col("date").alias("Date"),
    pl.col("start_time"),
    pl.col("destination"),
    pl.col("plugged_status"),
    pl.col("storage_type"),
    pl.col("end_time"),
    pl.col("overtime"),
    pl.col("total_tonnage"),
    pl.col("Price"),
    pl.col("invoice_value"),
).collect()
# wDATA.collect()

sys:1: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided


Date,start_time,destination,plugged_status,storage_type,end_time,overtime,total_tonnage,Price,invoice_value
date,time,str,enum,enum,time,str,f64,f64,f64
2025-02-12,08:25:00,"""SUDU8124408""","""Full""","""Brine""",10:04:00,"""normal hours""",26.94,25.5,686.97
2025-02-12,09:52:00,"""MNBU3009601""","""Partial""","""Brine""",14:19:00,"""normal hours""",22.69,25.5,578.595
2025-02-12,10:13:00,"""TLLU1164052""","""Full""","""Brine""",11:18:00,"""normal hours""",27.0,25.5,688.5
2025-02-12,11:33:00,"""MNBU3661013""","""Full""","""Brine""",12:18:00,"""normal hours""",26.97,25.5,687.735
2025-02-12,12:02:00,"""MNBU3681339""","""Full""","""Brine""",13:30:00,"""normal hours""",27.03,25.5,689.265
…,…,…,…,…,…,…,…,…,…
2025-02-14,09:57:00,"""MNBU9160277""","""Completed""","""Brine""",13:49:00,"""normal hours""",12.54,25.5,319.77
2025-02-14,09:57:00,"""SUDU6259554""","""Full""","""Brine""",11:09:00,"""normal hours""",26.64,25.5,679.32
2025-02-14,11:24:00,"""MNBU3779942""","""Full""","""Brine""",12:13:00,"""normal hours""",26.87,25.5,685.185


In [2]:
stuffingData.filter(
    pl.col("vessel_client").eq(pl.lit(vessel_name)),
    pl.col("date_plugged").is_between(start_date, end_date),
    pl.col("operation_type").str.contains(pl.lit("OSS")),
).with_columns(
    date_plugged=pl.col("date_plugged").dt.combine(pl.col("time_plugged"))
).select(pl.col("date_plugged"), pl.col("container_number")).collect()

date_plugged,container_number
datetime[μs],enum
2025-02-12 16:10:00,"""MMAU1272939"""
2025-02-12 14:36:00,"""MNBU3009601"""
2025-02-12 15:05:00,"""MNBU3217626"""
2025-02-12 13:23:00,"""MNBU3661013"""
2025-02-12 13:53:00,"""MNBU3681339"""
…,…
2025-02-14 14:01:00,"""MNBU3779942"""
2025-02-14 14:22:00,"""MNBU9160277"""
2025-02-14 09:20:00,"""MWCU5261126"""


In [3]:
from data.price import get_price

price = await get_price()

price.collect()

Service,Price,Date
str,f64,str
"""Additional Overtime""",7.5,"""01/01/2023"""
"""Additional Overtime""",8.0,"""01/03/2024"""
"""Basic OSS - Brine""",16.0,"""01/01/2023"""
"""Basic OSS - Brine""",18.25,"""01/03/2024"""
"""Basic OSS - Dry""",30.5,"""01/01/2023"""
…,…,…
"""Loading to Container - Brine""",22.5,"""01/01/2023"""
"""Loading to Container - Brine""",20.5,"""01/03/2024"""
"""Loading to Container - Dry""",23.5,"""01/01/2023"""


In [4]:
argi = "/home/garry-mounac/Dropbox/Crewing and Tally Clerk/Registers/2025/02 FEBRUARY 2025/REGISTER - 12.02.25.xlsm"

In [ ]:
pl.read_excel(argi, sheet_name="TXORI ARGI- 12.02.25").filter(
    pl.col("__UNNAMED__4").is_not_null(), pl.col("__UNNAMED__0").is_not_null()
).select(
    pl.col("__UNNAMED__0").alias("num"),
    pl.col("__UNNAMED__5")
    .str.replace("1899-12-31 ", "")
    .str.to_time(format="%H:%M:%S")
    .alias("end_time"),
).group_by(pl.col("end_time")).agg(pl.len().alias("num"))

end_time,num
time,u32
20:54:00,31
17:00:00,32


In [6]:
ops = "/home/garry-mounac/Dropbox/! OPERATION SUPPORTING DOCUMENTATION/2025/2025 IPHS operation activity.xlsx"

In [7]:
pl.read_excel(ops, sheet_name="HANDLING ACTIVITY").filter(
    pl.col("DATE").eq(date(2025, 2, 12)), pl.col("VESSEL NAME").eq(pl.lit("TXORI ARGI"))
).select(pl.col("Comments")).to_series().to_list()

['CCCS Front & Back / Extramen Sorting (12) + Well Support (10)']

In [2]:
import polars as pl
from dataframe import miscellaneous

await miscellaneous._cross_stuffing()